# Comparative Analysis

## Importando bibliotecas

In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, cross_validate, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder,
    LabelEncoder, SplineTransformer, OrdinalEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

## Tratamento dos Dados

#### Como nosso objetivo é predizer a precipitação mensal para as Regiões Hidrográficas do Estado do Ceará, precisamos que nosso modelo consiga fazer a predição da precipitação futura. <br><br> Para tal, foi criada uma função que cria uma janela e para cada ponto (lat,lon), adiciona a respectiva precipitação 2 meses à frente.  <br><br>  Além disso, avaliamos a influência das variáveis preditoras (índices oceânicos e variáveis atmosféricas) nos 4 meses anteriores, a fim de buscar a melhor forma de predição. 

In [3]:
def get_future(df, columns, janela):
    """
    A função pega a base de dados, e para cada ponto (lat,lon), adiciona a respectiva precipitação 2 meses à frente e algumas variáveis nos últimos 4 meses

    """
    suffix = 'mais' if janela > 0 else 'menos'
    df_out = df.copy()
    new_columns = [f'{variavel}_{suffix}_{abs(janela)}' for variavel in columns]
    for posicao in df.posicao.unique():
        criteria = "posicao == @posicao"
        df_out.loc[df_out.eval(criteria), new_columns] = (
            df_out
            .query(criteria)
            .shift(periods=-janela)[columns].values
        )
    return df_out

important_columns = [
    'EMI', 'nino3', 'atl3', 'seta' 
]
df_original = (
    pd
    #.read_csv("/content/drive/MyDrive/Pyoneers/data_regiao_hidro.csv") # para execução no colab
    .read_csv("../data/raw/data_regiao_hidro.csv")
    .pipe(get_future, ['pr'], 2)
    .pipe(get_future, important_columns, -1)
    .pipe(get_future, important_columns, -2)
    .pipe(get_future, important_columns, -3)
    .pipe(get_future, important_columns, -4)
    #.sample(1000, random_state=42) #remover depois
)

In [4]:
df_original

,data,posicao,pr,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,...,atl3_menos_1,EMI_menos_2,nino3_menos_2,atl3_menos_2,EMI_menos_3,nino3_menos_3,atl3_menos_3,EMI_menos_4,nino3_menos_4,atl3_menos_4
0,1981-01-01,"(-4.75, -39.25)",68.92,-2.849130e-06,74.14,-0.01,0.000007,93.54,0.49,-0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1981-01-01,"(-4.75, -39.0)",59.98,-3.877240e-06,73.94,-0.02,0.000008,120.53,0.49,-0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1981-01-01,"(-4.75, -38.75)",54.32,-4.451220e-06,73.52,-0.03,0.000011,75.55,0.49,-0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1981-01-01,"(-4.75, -38.5)",34.91,-4.331980e-06,73.19,-0.02,0.000014,24.54,0.49,-0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1981-01-01,"(-4.5, -39.0)",60.35,-2.912150e-06,74.28,0.07,0.000006,81.54,0.49,-0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96325,2022-02-01,"(-5.25, -40.25)",12.81,7.554250e-07,78.69,-0.04,0.000015,97.69,0.04,-1.17,...,0.23,-0.41,-1.28,0.23,-0.77,-1.03,0.55,-0.77,-0.79,0.65
96326,2022-02-01,"(-5.0, -40.75)",27.71,1.383160e-06,79.00,-0.13,0.000006,-135.32,0.04,-1.17,...,0.23,-0.41,-1.28,0.23,-0.77,-1.03,0.55,-0.77,-0.79,0.65
96327,2022-02-01,"(-5.0, -40.5)",24.11,-1.849470e-07,79.46,-0.09,-0.000002,120.67,0.04,-1.17,...,0.23,-0.41,-1.28,0.23,-0.77,-1.03,0.55,-0.77,-0.79,0.65
96328,2022-02-01,"(-5.0, -40.25)",16.98,2.199350e-06,78.71,-0.05,0.000014,292.69,0.04,-1.17,...,0.23,-0.41,-1.28,0.23,-0.77,-1.03,0.55,-0.77,-0.79,0.65


In [5]:
df = df_original.assign(
    lat = df_original.posicao.apply(lambda x: eval(x)[0]),
    lon = df_original.posicao.apply(lambda x: eval(x)[1]),
    ano = df_original.data.apply(lambda x: int(x[:4])),
    mes = df_original.data.apply(lambda x: int(x[5:7]))
).drop(columns=["data","posicao","regiao_hidro"], axis=1).dropna()

In [6]:
df

,pr,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,atn,ats,...,EMI_menos_3,nino3_menos_3,atl3_menos_3,EMI_menos_4,nino3_menos_4,atl3_menos_4,lat,lon,ano,mes
780,40.41,-3.356390e-06,85.50,-0.10,0.000006,51.78,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-39.25,1981,5
781,51.94,-2.265260e-06,84.89,-0.11,0.000005,72.78,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-39.00,1981,5
782,50.19,-1.520130e-06,84.39,-0.12,0.000004,82.79,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-38.75,1981,5
783,47.31,-2.272670e-06,83.94,-0.08,0.000003,15.77,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-38.50,1981,5
784,90.20,-1.694980e-06,85.09,-0.04,0.000004,22.76,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.50,-39.00,1981,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95935,25.01,-9.047460e-07,69.77,0.00,0.000017,153.35,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.25,-40.25,2021,12
95936,46.20,-8.485490e-08,70.93,-0.17,0.000004,-91.65,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.75,2021,12
95937,34.76,-1.224790e-06,70.94,-0.13,0.000006,-7.64,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.50,2021,12
95938,24.72,4.143700e-07,70.24,-0.04,0.000019,271.37,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.25,2021,12


In [7]:
df.isnull().sum().sort_values(ascending=False)

pr                0
nino3_menos_1     0
ano               0
lon               0
lat               0
atl3_menos_4      0
nino3_menos_4     0
EMI_menos_4       0
atl3_menos_3      0
nino3_menos_3     0
EMI_menos_3       0
atl3_menos_2      0
nino3_menos_2     0
EMI_menos_2       0
atl3_menos_1      0
EMI_menos_1       0
divergencia       0
pr_mais_2         0
nesta             0
seta              0
atl3              0
atlgrad           0
ats               0
atn               0
nino3             0
EMI               0
fluxo_energia     0
vorticidade       0
vento_vertical    0
umidade           0
mes               0
dtype: int64

In [8]:
target_column = 'pr_mais_2'
#nominal_columns = [column for column in list(df.select_dtypes(object)) if column != target_column]
quantitative_columns = [column for column in list(df.select_dtypes(np.number)) if column != 'pr_mais_2'] 

In [9]:
df[df.ano>2010]

,pr,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,atn,ats,...,EMI_menos_3,nino3_menos_3,atl3_menos_3,EMI_menos_4,nino3_menos_4,atl3_menos_4,lat,lon,ano,mes
70200,149.94,2.157950e-06,84.31,-0.10,0.000009,55.26,-1.47,-1.27,0.66,0.19,...,-1.63,-1.50,0.59,-1.42,-1.31,0.44,-4.75,-39.25,2011,1
70201,171.23,2.135090e-06,83.88,-0.11,0.000009,66.25,-1.47,-1.27,0.66,0.19,...,-1.63,-1.50,0.59,-1.42,-1.31,0.44,-4.75,-39.00,2011,1
70202,192.79,2.403860e-06,83.43,-0.11,0.000011,89.28,-1.47,-1.27,0.66,0.19,...,-1.63,-1.50,0.59,-1.42,-1.31,0.44,-4.75,-38.75,2011,1
70203,242.05,3.465330e-06,82.99,-0.10,0.000014,10.27,-1.47,-1.27,0.66,0.19,...,-1.63,-1.50,0.59,-1.42,-1.31,0.44,-4.75,-38.50,2011,1
70204,205.64,3.401690e-06,83.54,-0.03,0.000006,18.26,-1.47,-1.27,0.66,0.19,...,-1.63,-1.50,0.59,-1.42,-1.31,0.44,-4.50,-39.00,2011,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95935,25.01,-9.047460e-07,69.77,0.00,0.000017,153.35,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.25,-40.25,2021,12
95936,46.20,-8.485490e-08,70.93,-0.17,0.000004,-91.65,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.75,2021,12
95937,34.76,-1.224790e-06,70.94,-0.13,0.000006,-7.64,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.50,2021,12
95938,24.72,4.143700e-07,70.24,-0.04,0.000019,271.37,-0.41,-1.28,0.22,0.27,...,-0.59,-0.44,0.92,-0.42,-0.46,1.27,-5.00,-40.25,2021,12


In [10]:
X = df.drop(columns=[target_column], axis=1)
y = df[target_column].ravel()

In [13]:
print(X.shape)
print(y.shape)

(69420, 30)
(69420,)


In [14]:
X

,pr,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,atn,ats,...,EMI_menos_3,nino3_menos_3,atl3_menos_3,EMI_menos_4,nino3_menos_4,atl3_menos_4,lat,lon,ano,mes
780,40.41,-3.356390e-06,85.50,-0.10,5.810490e-06,51.78,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-39.25,1981,5
781,51.94,-2.265260e-06,84.89,-0.11,4.736120e-06,72.78,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-39.00,1981,5
782,50.19,-1.520130e-06,84.39,-0.12,3.604490e-06,82.79,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-38.75,1981,5
783,47.31,-2.272670e-06,83.94,-0.08,2.830210e-06,15.77,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.75,-38.50,1981,5
784,90.20,-1.694980e-06,85.09,-0.04,3.634270e-06,22.76,-0.51,-0.21,0.38,-0.33,...,0.26,-0.76,-0.16,0.49,-0.63,-0.32,-4.50,-39.00,1981,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70195,50.89,-4.568030e-07,78.54,0.04,1.277670e-05,104.39,-1.51,-1.51,0.71,0.14,...,-1.42,-1.31,0.44,-1.12,-1.15,0.33,-5.25,-40.25,2010,12
70196,91.52,8.505750e-07,77.60,-0.03,1.843810e-07,-54.61,-1.51,-1.51,0.71,0.14,...,-1.42,-1.31,0.44,-1.12,-1.15,0.33,-5.00,-40.75,2010,12
70197,84.36,-5.056130e-07,78.83,-0.03,3.493160e-07,128.41,-1.51,-1.51,0.71,0.14,...,-1.42,-1.31,0.44,-1.12,-1.15,0.33,-5.00,-40.50,2010,12
70198,60.09,1.994220e-06,78.86,0.03,1.140680e-05,121.42,-1.51,-1.51,0.71,0.14,...,-1.42,-1.31,0.44,-1.12,-1.15,0.33,-5.00,-40.25,2010,12


## Treinando os Modelos

#### Os dados passaram por um pré-processamento prévio ao treinamento, onde foram padronizados, de forma a estarem na mesma magnitude, e a sazonalidade dos meses foi considerada, por meio da aplicação dos métodos: One-Hot Encoding, transformção usando seno e cosseno e spline. <br><br> Os  modelos escolhidos foram:
- **Regressão Linear**;
- **Regressão Ridge**;
- **Regressão Lasso**;
- **KNN**;
- **Árvore de Decisão**;
- **Xgboost**.

In [15]:
def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

In [16]:
def sin_transformer(x):
    return np.sin(x / 12 * 2 * np.pi)

def cos_transformer(x):
    return np.cos(x / 12 * 2 * np.pi)

In [17]:
# nominal_preprocessing = Pipeline([    
#     ("missing", SimpleImputer(strategy='most_frequent')),
#     ("encoder", OneHotEncoder(sparse=False)),
#     ("scaler", StandardScaler())
# ])

quantitative_preprocessing = Pipeline([
    ("missing", SimpleImputer()),
    ("scaler", StandardScaler())
])
preprocessing = ColumnTransformer([
    ("month_sin", FunctionTransformer(sin_transformer), ["mes"]),
    ("month_cos", FunctionTransformer(cos_transformer), ["mes"]),
    #("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["mes"]),
    #("ohe", OneHotEncoder(), ["mes"]),
    #("nominal", nominal_preprocessing, nominal_columns),
    ("quantitative", quantitative_preprocessing, quantitative_columns)
])

In [18]:
models = [{
    'name': 'KNN',
    'model': KNeighborsRegressor(algorithm='kd_tree'),
    'parameters': {
        'n_neighbors': np.arange(3, 17, 2),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan'],
        'leaf_size': [10, 20, 30, 40, 50, 60, 80]
    }
},{
    'name': 'Linear',
    'model': LinearRegression(),
    'parameters': {
       }
},{
    'name': 'Lasso',
    'model': Lasso(),
    'parameters': {
        'alpha': [1, 0.8, 0.5, 0.3, 0.2, 0.1, 0.05, 0.025],
        'fit_intercept': [True, False],
       }
},{
    'name': 'Ridge',
    'model': Ridge(random_state=42),
    'parameters': {
        'alpha': [1, 0.8, 0.5, 0.3, 0.2, 0.1, 0.05, 0.025],
        'fit_intercept': [True, False],
        'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
       }
},{
    'name': 'XGBoost',
    'model': XGBRegressor(random_state=42),
    'parameters': {
        'learning_rate': [0.05, 0.1, 0.2],
        'n_estimators' : [100, 200, 300, 500, 1000],
        'max_depth'    : [3, 5, 8, 10, 20, 30, 40],
        'min_child_weight': [1, 3, 5, 10]
    }
},{
    'name': 'Decision Tree',
    'model':  DecisionTreeRegressor(random_state=42),
    'parameters': {
        "splitter":["best","random"],
        "max_depth" : [3,5,8,10,15,20],
        "max_leaf_nodes":[None,10,20,30,40,50,60,70,80],
        "min_samples_leaf": np.linspace(0.1, 0.5, 6),
        "max_features": ["log2", "sqrt"],
        "criterion": ["friedman_mse", "squared_error"]
    }
}]

In [19]:
def concatenate(*args):
    final_dict = {key: [] for key in args[0].keys()}
    for dictionary in args:
        for key, value in dictionary.items():
            final_dict[key].extend(value)
    return final_dict

## Avaliando os modelos

#### As métricas utilizados para avaliar o desempenho dos modelos de regressão foram:

- **MAE** (Erro Absoluto Médio): calcula o "erro absoluto médio" dos erros entre valores observados (reais) e predições (hipóteses). Quanto maior seu valor, pior o desempenho do modelo.
- **MSE** (Erro quadrado médio): Mede o erro ao quadrado médio das previsões do modelo. MSE calcula a diferença ao quadrado entre o resultado observado (real) e os valores previstos e depois calcula a média. Quanto maior seu valor, pior o desempenho do modelo.
- **RMSE** (Raiz do Erro Quadrático Médio): Representa a raiz quadrada do MSE. O RMSE mede a diferença entre os valores previstos pelo modelo e os valores observados (reais). <br> Quanto maior seu valor, pior o desempenho do modelo.
- **R²**: expressa a porcentagem de variância explicada pelas variáveis independentes apresentadas no modelo. Quando maior, melhor o desempenho dele. <br> 

In [ ]:
n_splits_cv = 5
n_splits_cv_gs = 5
sc = []
for model in models:
    print(f"running {model['name']}")
    param_grid = {
        'preprocessing__quantitative__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
        #'preprocessing__nominal__encoder': [OneHotEncoder(sparse=False), OrdinalEncoder()],
        #'preprocessing__nominal__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
        'preprocessing__quantitative__missing__strategy': ['mean', 'median'],
        **{f"model__{key}": value for key, value in model['parameters'].items()}
    }
    approach = Pipeline([
        ('preprocessing', preprocessing),
        ('model', model['model'])
    ])
    gs = RandomizedSearchCV(
        estimator=approach,
        param_distributions=param_grid,
        scoring='r2',
        cv=n_splits_cv_gs,
        random_state=42
    )
    scores = cross_validate(
        estimator = gs,
        X=X,
        y=y,
        cv = n_splits_cv,
        n_jobs = -1,
        scoring = [
            'neg_mean_absolute_error',
            'neg_mean_squared_error',
            'neg_root_mean_squared_error',
            'r2'
        ]
    )
    scores['model'] = [model['name']] * n_splits_cv
    sc.append(scores)
scores = concatenate(*sc)

In [21]:
def highlight_max(s, props=''):
    values = [float(value.split()[0]) for value in s.values[1:]]
    result = [''] * len(s.values)
    if s.values[0].endswith('time'):
        result[np.argmin(values)+1] = props
    else:
        result[np.argmax(values)+1] = props
    return result

def get_winner(s):
    metric = s.values[0]
    values = [float(value.split()[0]) for value in s.values[1:]]
    models = results.columns[1:]
    
    if s.values[0].endswith('time'):
        return models[np.argmin(values)]
    else:
        return models[np.argmax(values)]

results = (
    pd
    .DataFrame(scores)
    .groupby(['model'])
    .agg([lambda x: f"{np.mean(x):.3f} ± {np.std(x):.3f}"])#
    .transpose()
    .reset_index()
    .rename(columns={"level_0": "score"})
    .drop(columns="level_1")
    # .set_index('score')
)
time_scores = ['fit_time', 'score_time']
winner = results.query('score not in @time_scores').apply(get_winner, axis=1).value_counts().index[0]
results.columns.name = ''
results = (
    results
    .style
    #.hide(axis='index')
    .apply(highlight_max, props='color:white;background-color:gray', axis=1)
)
display(results)
print(f'O melhor modelo é o {winner}')

,score,Decision Tree,KNN,Lasso,Linear,Ridge,XGBoost
0,fit_time,6.223 ± 0.108,1622.146 ± 24.420,44.160 ± 1.145,7.898 ± 0.050,42.960 ± 2.125,3047.922 ± 32.434
1,score_time,0.011 ± 0.002,11.986 ± 1.064,0.019 ± 0.007,0.014 ± 0.005,0.012 ± 0.002,0.067 ± 0.016
2,test_neg_mean_absolute_error,-50.908 ± 4.417,-36.351 ± 4.028,-41.210 ± 2.587,-42.365 ± 1.157,-42.425 ± 0.843,-34.706 ± 2.733
3,test_neg_mean_squared_error,-5261.565 ± 1200.632,-4335.065 ± 1018.068,-3746.038 ± 770.254,-3663.201 ± 507.538,-3674.659 ± 498.926,-2994.567 ± 448.557
4,test_neg_root_mean_squared_error,-72.054 ± 8.350,-65.305 ± 8.390,-60.896 ± 6.140,-60.377 ± 4.222,-60.477 ± 4.152,-54.555 ± 4.280
5,test_r2,0.359 ± 0.071,0.453 ± 0.167,0.542 ± 0.034,0.546 ± 0.045,0.545 ± 0.047,0.622 ± 0.088


O melhor modelo é o XGBoost


In [ ]:
best_model = next(item for item in models if item["name"] == winner)

param_grid = {
    'preprocessing__quantitative__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'preprocessing__quantitative__missing__strategy': ['mean', 'median'],
    **{f"model__{key}": value for key, value in best_model['parameters'].items()}
}

approach = Pipeline([
    ('preprocessing', preprocessing),
    ('model', best_model['model'])
])

gs = RandomizedSearchCV(
    estimator=approach,
    param_distributions=param_grid,
    scoring='r2',
    cv=n_splits_cv_gs,
    random_state=42
)

gs.fit(X, y)

model = gs.best_estimator_
joblib.dump(model, '../models/best_model_pr_mais_2.joblib')

### Considerações Finais
- Os melhores resultados foram obtidos considerando os índices EMI, seta, nino3 (indices oceanicos relacionados ao El Nino) e o Atl3 (Indice oceanico associado a variabilidade do Atlantico Tropical) dos 4 meses anteriores.
- Dentre os modelos utilizados, o melhor resultado foi obtido pelo modelo Xgboost. Com RMSE próximo a 54 mm e com as variáveis preditoras explicando mais de 62% da variabilidade da precipitação 2 meses no futuro.